<a href="https://colab.research.google.com/github/Anthonylwz/cssma/blob/main/banco2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install yfinance matplotlib seaborn scikit-learn tensorflow plotly gradio


In [22]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

import yfinance as yf
import gradio as gr

# Configuración
TICKERS = ["TSLA", "NVDA"]
SEQ_LEN = 60   # longitud de la secuencia LSTM
MODELS_DIR = "./saved_models"
os.makedirs(MODELS_DIR, exist_ok=True)

print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.19.0


In [23]:
def fetch_intraday_yf(symbol, interval="1m", period="1d"):
    """
    Descarga datos intradía desde Yahoo Finance (ej. 1m del último día).
    Ajusta la zona horaria automáticamente.
    """
    df = yf.download(symbol, interval=interval, period=period, progress=False)

    # Fix timezone: si ya tiene tz, convertir, si no, asignar
    if df.index.tz is None:
        df.index = df.index.tz_localize("UTC").tz_convert("America/Lima")
    else:
        df.index = df.index.tz_convert("America/Lima")

    return df


In [24]:
def prepare_sequences_from_series(series, seq_len=SEQ_LEN):
    arr = series.values.reshape(-1,1).astype(float)
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(arr)
    Xs, ys = [], []
    for i in range(len(scaled)-seq_len):
        Xs.append(scaled[i:i+seq_len])
        ys.append(scaled[i+seq_len])
    return np.array(Xs), np.array(ys), scaler

def build_lstm_model(seq_len, units=128, dropout=0.3):
    model = Sequential([
        LSTM(units, return_sequences=True, input_shape=(seq_len,1)),
        Dropout(dropout),
        LSTM(units),
        Dropout(dropout),
        Dense(1)
    ])
    model.compile(optimizer="adam", loss="mse")
    return model


In [29]:
def predict_realtime_yf(ticker, epochs=20, batch_size=32, robust=False):
    # Descargar intradía en tiempo real
    df_intraday = fetch_intraday_yf(ticker, interval="1m", period="1d")
    series = df_intraday["Close"].astype(float)

    # Preparar datos
    X_intr, y_intr, scaler_intr = prepare_sequences_from_series(series, seq_len=SEQ_LEN)
    if len(X_intr) < 10:
        return f"Datos insuficientes para {ticker}.", go.Figure(), df_intraday.tail(10)

    # Construcción del modelo
    model = build_lstm_model(SEQ_LEN, units=128, dropout=0.3)

    # Epochs robusto si se selecciona
    train_epochs = 5000 if robust else epochs

    model.fit(X_intr, y_intr, epochs=train_epochs, batch_size=batch_size, verbose=0)

    # Predicción próxima vela
    last_seq = X_intr[-1].reshape(1, SEQ_LEN, 1)
    pred_scaled = model.predict(last_seq)
    next_price = scaler_intr.inverse_transform(pred_scaled)[0][0]
    last_price = float(series.iloc[-1])   # 🔹 aseguramos que es escalar
    trend = "📈 SUBE" if next_price > last_price else "📉 BAJA"

    # Gráfico
    recent = series.tail(120)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=recent.index, y=recent.values, mode="lines", name=ticker))
    fig.add_trace(go.Scatter(
        x=[recent.index[-1], recent.index[-1]+pd.Timedelta(minutes=1)],
        y=[last_price, next_price],
        mode="markers+lines", name="Predicción"
    ))
    fig.update_layout(title=f"{ticker} - Intradía (1m) + Predicción",
                      xaxis_title="Hora (Perú)", yaxis_title="USD")

    text = f"""
    Último precio: {last_price:.2f} USD
    Predicción próximo minuto: {next_price:.2f} USD → {trend}
    (Entrenamiento con {train_epochs} epochs)
    """

    return text.strip(), fig, df_intraday.tail(10)



In [30]:
def gradio_handler(ticker, epochs, batch_size, robust):
    try:
        text, fig, table = predict_realtime_yf(
            ticker, epochs=int(epochs), batch_size=int(batch_size), robust=robust
        )
        return text, fig, table
    except Exception as e:
        return f"Error: {e}", None, None

with gr.Blocks() as demo:
    gr.Markdown("# 🔮 Predicción EN TIEMPO REAL (TSLA vs NVDA) con yfinance")
    with gr.Row():
        with gr.Column(scale=1):
            ticker = gr.Dropdown(choices=TICKERS, value="TSLA", label="Ticker")
            epochs = gr.Number(value=20, label="Epochs (entrenamiento rápido)", precision=0)
            batch = gr.Number(value=32, label="Batch Size", precision=0)
            robust = gr.Checkbox(label="Entrenamiento robusto (5000 epochs)", value=False)
            run_btn = gr.Button("Ejecutar Predicción")
        with gr.Column(scale=2):
            out_text = gr.Textbox(label="Resultados")
            out_plot = gr.Plot(label="Gráfico intradía + Predicción")
            out_table = gr.Dataframe(label="Últimos precios intradía")

    run_btn.click(fn=gradio_handler,
                  inputs=[ticker, epochs, batch, robust],
                  outputs=[out_text, out_plot, out_table])

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2c7a56b2e59dc642d1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
